In [7]:
import platform
import torch

def showinfo(tip, info):
    print("{}:{}".format(tip,info))

showinfo("操作系统及版本信息",platform.platform())
showinfo('系统位数', platform.architecture())
showinfo('pytorch版本', torch.__version__)
showinfo('cuda版本', torch.version.cuda)
showinfo('cudnn版本', torch.backends.cudnn.version())

操作系统及版本信息:Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
系统位数:('64bit', 'ELF')
pytorch版本:2.3.0
cuda版本:12.1
cudnn版本:8902


## 卷积神经网络

### cv界的神
通过加入卷积层，用于提取图片特征，进而有效的进行图像识别。

## 基于pytorch直接定义一个cnn卷积神经网络

In [8]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 加载mnist数字识别数据集

In [9]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = MNIST(root='./data', train=True, download=True, transform=transform)
test_set = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [10]:
## 定义训练过程

- epochs = 5：训练五次
- lr = 0.001：学习率
- 损失函数：交叉熵
- 优化方法：adam梯度下降

SyntaxError: cannot assign to operator (1035591872.py, line 3)

In [11]:
import torch.optim as optim


lr = 0.001
epochs = 5

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        ## 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        ## 反向传播
        optimizer.zero_grad() # 清楚上次训练的遗留梯度信息
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'cnn_mnist.pth')

Epoch [1/5], Step [100/938], Loss: 0.1588
Epoch [1/5], Step [200/938], Loss: 0.1087
Epoch [1/5], Step [300/938], Loss: 0.0991
Epoch [1/5], Step [400/938], Loss: 0.1589
Epoch [1/5], Step [500/938], Loss: 0.1282
Epoch [1/5], Step [600/938], Loss: 0.0249
Epoch [1/5], Step [700/938], Loss: 0.0940
Epoch [1/5], Step [800/938], Loss: 0.0114
Epoch [1/5], Step [900/938], Loss: 0.1104
Epoch [2/5], Step [100/938], Loss: 0.0145
Epoch [2/5], Step [200/938], Loss: 0.0278
Epoch [2/5], Step [300/938], Loss: 0.0885
Epoch [2/5], Step [400/938], Loss: 0.0164
Epoch [2/5], Step [500/938], Loss: 0.0089
Epoch [2/5], Step [600/938], Loss: 0.0046
Epoch [2/5], Step [700/938], Loss: 0.0438
Epoch [2/5], Step [800/938], Loss: 0.1579
Epoch [2/5], Step [900/938], Loss: 0.0082
Epoch [3/5], Step [100/938], Loss: 0.0117
Epoch [3/5], Step [200/938], Loss: 0.0255
Epoch [3/5], Step [300/938], Loss: 0.0179
Epoch [3/5], Step [400/938], Loss: 0.0043
Epoch [3/5], Step [500/938], Loss: 0.0014
Epoch [3/5], Step [600/938], Loss:

In [15]:
model = CNN()
model.load_state_dict(torch.load('cnn_mnist.pth'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy on the test set: {100 * correct / total}%')

Accuracy on the test set: 99.15%
